# Comprehensive SessionManager Demo

This notebook demonstrates comprehensive Neptune Analytics instance management using SessionManager with automatic cleanup.

This notebook covers:
1. Start/stop instances
2. Reset instance
3. Export an instance to S3
4. Create a snapshot from an instance
5. Create multiple instances in parallel (empty, from S3, from snapshot)
6. Automatic cleanup using context manager (__exit__)

## Setup

Import the necessary libraries and set up logging.

In [1]:
import logging
import sys
import os
import asyncio
from pprint import pprint

import dotenv
dotenv.load_dotenv()

from nx_neptune.session_manager import SessionManager, CleanupTask
from nx_neptune import instance_management
import networkx as nx

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Environment Variables

Set up required environment variables for S3 operations.

In [3]:
# Required environment variables
s3_import_bucket = os.getenv('NETWORKX_S3_IMPORT_BUCKET_PATH')
s3_export_bucket = os.getenv('NETWORKX_S3_EXPORT_BUCKET_PATH')
iam_role_arn = os.getenv('NETWORKX_ARN_IAM_ROLE')

print(f"S3 Import Bucket: {s3_import_bucket}")
print(f"S3 Export Bucket: {s3_export_bucket}")
print(f"IAM Role ARN: {iam_role_arn}")

if not all([s3_import_bucket, s3_export_bucket, iam_role_arn]):
    print("⚠️ Warning: Some environment variables are not set. S3 operations may fail.")

S3 Import Bucket: s3://nx-cit-patents/csv-import/
S3 Export Bucket: s3://nx-cit-patents/csv-export/
IAM Role ARN: None
⚠️ Warning: Some environment variables are not set. S3 operations may fail.


## Scenario 1: Start/Stop Instances

Create an instance, stop it, then start it again using SessionManager.

In [5]:
print("=== Scenario 1: Start/Stop Instances ===")

with SessionManager("start-stop-demo") as session:
    # Create a new instance
    print("Creating new instance...")
    graph = await session.get_or_create_graph()
    print(f"Created instance: {graph}")
    
    # Stop the instance
    print("Stopping instance...")
    await session.stop_graph(graph)
    print("Instance stopped")
    
    # Start the instance again
    print("Starting instance...")
    await session.start_graph(graph)
    print("Instance started")
    
    # List graphs to verify status
    graphs = session.list_graphs()
    print(f"Current graphs: {len(graphs)}")
    for graph in graphs:
        print(f"  - {graph['name']}: {graph['status']}")

print("✅ Scenario 1 completed - instances automatically destroyed")

=== Scenario 1: Start/Stop Instances ===
Creating new instance...


INFO - Creating new graph named with prefix: start-stop-demo
WARNING - No graphs found matching name: ['start-stop-demo'] and status: AVAILABLE


AttributeError: 'IAM' object has no attribute 'has_create_na_permissions'

## Scenario 2: Reset Instance

Create an instance, add some data, then reset it.

In [ ]:
print("=== Scenario 2: Reset Instance ===")

with SessionManager("reset-demo") as session:
    # Create a new instance
    print("Creating new instance...")
    graph_id = await instance_management.create_na_instance()
    print(f"Created instance: {graph_id}")
    
    # Add some sample data
    print("Adding sample data...")
    G = nx.Graph(backend="neptune")
    G.graph_id = graph_id
    G.add_edge("Alice", "Bob")
    G.add_edge("Bob", "Charlie")
    print(f"Added {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    
    # Reset the instance
    print("Resetting instance...")
    await session.reset_graph(graph_id)
    print("Instance reset completed")
    
    # Verify reset
    G_reset = nx.Graph(backend="neptune")
    G_reset.graph_id = graph_id
    print(f"After reset: {G_reset.number_of_nodes()} nodes and {G_reset.number_of_edges()} edges")

print("✅ Scenario 2 completed - instances automatically destroyed")

## Scenario 3: Export Instance to S3

Create an instance with data and export it to S3.

In [ ]:
print("=== Scenario 3: Export Instance to S3 ===")

if s3_export_bucket:
    with SessionManager("export-demo") as session:
        # Create a new instance
        print("Creating new instance...")
        graph_id = await instance_management.create_na_instance()
        print(f"Created instance: {graph_id}")
        
        # Add sample data
        print("Adding sample data...")
        G = nx.Graph(backend="neptune")
        G.graph_id = graph_id
        G.add_edge("Node1", "Node2")
        G.add_edge("Node2", "Node3")
        G.add_edge("Node3", "Node4")
        print(f"Added {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
        
        # Export to S3
        print(f"Exporting to S3: {s3_export_bucket}")
        task_id = await instance_management.export_csv_to_s3(G, s3_export_bucket)
        print(f"Export completed with task ID: {task_id}")
else:
    print("⚠️ Skipping S3 export - NETWORKX_S3_EXPORT_BUCKET_PATH not set")

print("✅ Scenario 3 completed - instances automatically destroyed")

## Scenario 4: Create Snapshot from Instance

Create an instance, add data, create a snapshot, then use the snapshot.

In [ ]:
print("=== Scenario 4: Create Snapshot from Instance ===")

with SessionManager("snapshot-demo") as session:
    # Create a new instance
    print("Creating source instance...")
    source_graph_id = await instance_management.create_na_instance()
    print(f"Created source instance: {source_graph_id}")
    
    # Add sample data
    print("Adding sample data...")
    G = nx.Graph(backend="neptune")
    G.graph_id = source_graph_id
    G.add_edge("A", "B")
    G.add_edge("B", "C")
    G.add_edge("C", "D")
    G.add_edge("D", "A")
    print(f"Added {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    
    # Create snapshot
    print("Creating snapshot...")
    snapshot_id = await instance_management.create_graph_snapshot(source_graph_id)
    print(f"Created snapshot: {snapshot_id}")
    
    # Create new instance from snapshot
    print("Creating instance from snapshot...")
    target_graph_id = await instance_management.create_na_instance_from_snapshot(snapshot_id)
    print(f"Created target instance: {target_graph_id}")
    
    # Verify data in new instance
    G_new = nx.Graph(backend="neptune")
    G_new.graph_id = target_graph_id
    print(f"Target instance has {G_new.number_of_nodes()} nodes and {G_new.number_of_edges()} edges")
    
    # Clean up snapshot
    print("Deleting snapshot...")
    await instance_management.delete_graph_snapshot(snapshot_id)
    print("Snapshot deleted")

print("✅ Scenario 4 completed - instances automatically destroyed")

## Scenario 5: Create Multiple Instances in Parallel

Create three instances in parallel: empty, from S3, and from snapshot.

In [ ]:
print("=== Scenario 5: Create Multiple Instances in Parallel ===")

with SessionManager("parallel-demo") as session:
    # First, create a source instance and snapshot for the third parallel creation
    print("Preparing snapshot for parallel creation...")
    prep_graph_id = await instance_management.create_na_instance()
    
    # Add some data to the prep instance
    G_prep = nx.Graph(backend="neptune")
    G_prep.graph_id = prep_graph_id
    G_prep.add_edge("X", "Y")
    G_prep.add_edge("Y", "Z")
    
    # Create snapshot
    snapshot_id = await instance_management.create_graph_snapshot(prep_graph_id)
    print(f"Created snapshot: {snapshot_id}")
    
    # Clean up prep instance
    await instance_management.delete_na_instance(prep_graph_id)
    
    # Now create three instances in parallel
    print("Creating three instances in parallel...")
    
    async def create_empty_instance():
        graph_id = await instance_management.create_na_instance()
        print(f"Empty instance created: {graph_id}")
        return graph_id
    
    async def create_s3_instance():
        if s3_import_bucket:
            graph_id, task_id = await instance_management.create_na_instance_with_s3_import(s3_import_bucket)
            print(f"S3 instance created: {graph_id} (task: {task_id})")
            return graph_id
        else:
            print("⚠️ Skipping S3 instance - NETWORKX_S3_IMPORT_BUCKET_PATH not set")
            return None
    
    async def create_snapshot_instance():
        graph_id = await instance_management.create_na_instance_from_snapshot(snapshot_id)
        print(f"Snapshot instance created: {graph_id}")
        return graph_id
    
    # Create all instances in parallel
    results = await asyncio.gather(
        create_empty_instance(),
        create_s3_instance(),
        create_snapshot_instance(),
        return_exceptions=True
    )
    
    print(f"Parallel creation completed. Created {len([r for r in results if r and not isinstance(r, Exception)])} instances")
    
    # List all graphs
    graphs = session.list_graphs()
    print(f"Total graphs in session: {len(graphs)}")
    for graph in graphs:
        print(f"  - {graph['name']}: {graph['status']}")
    
    # Clean up snapshot
    print("Deleting snapshot...")
    await instance_management.delete_graph_snapshot(snapshot_id)
    print("Snapshot deleted")

print("✅ Scenario 5 completed - all instances automatically destroyed")

## Scenario 6: Automatic Cleanup Demonstration

Show how the context manager automatically cleans up resources.

In [ ]:
print("=== Scenario 6: Automatic Cleanup Demonstration ===")

# Create a session manager without context manager to show manual cleanup
session = SessionManager("cleanup-demo")

try:
    # Create some instances
    print("Creating instances without context manager...")
    graph_id1, graph_id2 = await session.create_multiple_instances(2)

    graphs = session.list_graphs()
    print(f"Created {len(graphs)} instances")
    
    # Manually trigger cleanup using __exit__
    print("Manually triggering cleanup...")
    session.__exit__(None, None, None)
    
    print("Manual cleanup completed")
    
except Exception as e:
    print(f"Error: {e}")
    # Ensure cleanup even on error
    session.__exit__(type(e), e, e.__traceback__)

print("\n=== Using Context Manager for Automatic Cleanup ===")

# Demonstrate automatic cleanup with context manager
with SessionManager("auto-cleanup-demo") as session:
    print("Creating instances with context manager...")
    graph_id1, graph_id2 = await session.create_multiple_instances(2)

    graphs = session.list_graphs()
    print(f"Created {len(graphs)} instances")
    print("Exiting context manager - automatic cleanup will occur...")

print("✅ Scenario 6 completed - automatic cleanup demonstration finished")

## Summary

This notebook demonstrated comprehensive Neptune Analytics instance management using SessionManager:

1. **Start/Stop Instances**: Created, stopped, and restarted instances
2. **Reset Instance**: Added data and reset an instance to empty state
3. **Export to S3**: Exported graph data to S3 bucket
4. **Snapshot Operations**: Created snapshots and instances from snapshots
5. **Parallel Creation**: Created multiple instances simultaneously
6. **Automatic Cleanup**: Used context manager for automatic resource cleanup

Key benefits of using SessionManager:
- **Automatic Resource Management**: Context manager ensures cleanup
- **Session Isolation**: Session names provide resource grouping
- **Simplified API**: High-level operations for common tasks
- **Error Safety**: Resources cleaned up even on exceptions

All instances created during these scenarios were automatically destroyed when exiting the context manager, ensuring no resources were left running.